In [4]:
!pip install -q sentence-transformers dspy-ai gradio PyPDF2 transformers chromadb langchain langchain_community pypdf

In [7]:
import chromadb
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

chroma_client = chromadb.PersistentClient(path="db/")
collection = chroma_client.get_or_create_collection(name="city_laws", embedding_function=sentence_transformer_ef)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/Users/sakuard/dev/bootcamp_ai/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./dataset/DMC-Act-1957_0.pdf')
documents = pdf_loader.load_and_split()
print(len(documents))

167


In [11]:
docs = []
ids = []
i = 0

for doc in documents:
  docs.append(doc.page_content)
  ids.append(str(i))
  i = i+1

In [12]:
collection = chroma_client.get_or_create_collection(name="city_laws", embedding_function=sentence_transformer_ef)

collection.add(
    documents = docs,
    ids = ids
)

# DSPy

In [23]:
import dspy
# from dspy.retrieve.chromadb_rm import ChromadbRM

# retriever_model = ChromadbRM(
# 'city_laws', 'db/', embedding_function=sentence_transformer_ef, k=5
# )

lm = dspy.OllamaLocal(model='llama3')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# dspy.settings.configure(lm=lm, rm=retriever_model)
dspy.settings.configure(lm=lm, rm = colbertv2_wiki17_abstracts)

In [24]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [25]:
uncompiled_rag = RAG()
resp = uncompiled_rag("What are the laws in Delhi around pets?")

In [26]:
print(f"""Predicted Answer: {resp.answer}""")
print(f"""Retrieved Contexts (truncated): {[c for c in resp.context]}""")

Predicted Answer: Here is the completed format:

Context: [1] «Pet Fed India | Pet Fed is the biggest pet festival held in India, main edition held annually at New Delhi and Mumbai. For the first time, Delhi edition was organized on 1 November 2014 at Dilli Haat INA.» [2] «Animal welfare and rights in India | Animal welfare and rights in India regards the treatment of and laws concerning non-human animals in India. It is distinct from animal conservation in India.» [3] «Dangerous Wild Animals Act 1976 | The Dangerous Wild Animals Act 1976 (22 July) is a law of the United Kingdom that was originally enacted to deal with the increasing fashion in the late 1960s and early
Retrieved Contexts (truncated): ['Pet Fed India | Pet Fed is the biggest pet festival held in India, main edition held annually at New Delhi and Mumbai. For the first time, Delhi edition was organized on 1 November 2014 at Dilli Haat INA.', 'Animal welfare and rights in India | Animal welfare and rights in India regards 

In [29]:
!pip install dspy-ai[

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
# import os
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# # Set up the ChromadbRM client
# # embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name="text-embedding-ada-002")
# retriever_model = ChromadbRM('docs', './db', embedding_function="llama3", k=5)

# # Configure DSPy settings
# dspy.settings.configure(rm=retriever_model)

# # Define the Retrieve module
# retriever = dspy.Retrieve(k=5)

# # Query the retrieval model
# query = "Explore the significance of quantum computing"
# results = retriever(query).passages

# # Print the results
# for result in results:
#     print("Document:", result.long_text, "\n")


NameError: name 'dspy' is not defined